<h2>This script is for internal developer testing.</h2>

In [1]:
import wfdb

In [1]:
import wfdb
# Testing rdsamp multiple samples per frame (and fmt 12)
sig, fields=wfdb.srdsamp('sampledata/03700181')
print(sig[:20,:])
print("\n\n", sig.shape, "\n\n")
print(fields)

print("\n\n\n")
print(sig[-20:,:])

filename:  03700181.dat
dirname:  sampledata
pbdir:  None


UnboundLocalError: local variable 'nbytesload' referenced before assignment

In [ ]:
# Testing rdsamp format 80
sig, fields=wfdb.srdsamp('sampledata/3000003_0003')
print(sig)
print(fields)

In [ ]:
# Testing rdsamp format 16 with byte offset (actually a .mat file too)
sig, fields=wfdb.srdsamp('sampledata/a103l') 
#print(sig)
#print(sig.shape)
#print(fields)

from wfdb import plotwfdb

plotwfdb.plotsigs(sig, fields)


# With channel selection and sampfrom sampto
#sig, fields=readsignal.rdsamp('sampledata/a103l', channels=[2,0], sampfrom=250, sampto=80000) 
#print('\n\n')
#print(sig)
#print(fields)

In [ ]:
# For writing test annotation files based on first 3 annotations of 12726.anI

f=open('tmp.fake', 'wb')

indata= np.array([0, 236,   1,   0, 200,  84,   0,  88, 255, 251,  21, 252,  73, 110, 105, 116,
                  105,  97, 116, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 117,
                  112,   0,   0, 236,   0,   0,  67,  50,   0,  88,  21, 252,  67, 111, 110,  99,
108, 117, 100, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 117,
112,   0,   0, 236,   0,   0, 114, 183,   0,  88,  23, 252,  73, 110, 105, 116,
105,  97, 116, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 100,
111, 119, 110,   0, 0 ,0 ]).astype('u1')


print(indata)

indata.tofile(f)


f.close()


In [ ]:
# Trying to fix byte pointer for reading skewed signals. 

from wfdb import readsignal
import numpy as np
#sig, fields=readsignal.rdsamp('sampledata/03700181', channels=[0,2], sampfrom=1000, sampto=16000)
#sig=np.round(sig, decimals=8)
#targetsig=np.genfromtxt('tests/targetoutputdata/target10')

sig, fields=readsignal.rdsamp('sampledata/03700181', sampfrom=1000, sampto=16000, physical=0) # fs=125. -f 8 -t 128

print("\nsig: ", sig)
print('\n\n')
print("End of sig: ", sig[-10:,:])


# nbytesread = 15000 frames * 6 samples/frame * 1.5 bytes/sample = 135000

# Try to octal dump the right section of the file. After 16000 frames x (6 samples/frame)  = 96000 samples * (1.5 bytes per sample)
# = 144000 bytes skip. 



# od -t u1 03700181.dat -j 144000 -N 30 

# Result: 
# 236 255 236   2 240 236 163  27  61 236 255 236 236  15   2 147
#  27  40 236 255 236 236 255 236 130  27  19   2 240 236




In [ ]:
import wfdb
ann = wfdb.rdann('/home/chen/Downloads/chb01_03.edf','seizures')

In [ ]:
ann.__dict__

In [ ]:
# Testing wrann
import wfdb
from IPython.display import display

annsamp = [20, 40, 60, 80, 100, 800, 1200 , 3000, 3001, 8000, 100000]
#annsamp = [20, 40, 60, 80, 100, 800, 1200]
#annsamp = [20, 60, 2**31] # 2**31+100 is too large

nsamp = len(annsamp)

anntype = ['N', 'N', 'N', '3', 'N', '2', 'N', '1', 'N', 'N', 'g']
sub = [1,0,0,0,5,3,2,5,0,0,0]
chan = [2,2,3,2,9,2,2,2,2,2,2]
num = [1,2,3,4,5,3,2,5,11,22,22]
aux = [None, None, "good", "good", "good", "good", "good", "good", None, None, None]
fs = 100
fs = None

a = wfdb.Annotation(recordname = 'firstann', annotator = 'atr', 
                    annsamp = annsamp, anntype = anntype, aux = aux, 
                    subtype = sub, chan = chan, num = num, fs = fs)
a.type2aux()
display(a.__dict__)
a.wrann()

b = wfdb.rdann('firstann', 'atr')
b.__dict__


# print([sd & 255, sd & 768, (sd & 768) >> 8])

In [4]:
import requests
from IPython.display import display
url = 'http://physionet.org/physiobank/database/mitdb/100.hea'
r = requests.get(url)
r

<Response [200]>

In [5]:
dir(r)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [9]:
display(r.url)
display(r.raw)
display(r.content)
display(r.headers)
display(type(r.content))

'http://physionet.org/physiobank/database/mitdb/100.hea'

b'100 2 360 650000\r\n100.dat 212 200 11 1024 995 -22131 0 MLII\r\n100.dat 212 200 11 1024 1011 20052 0 V5\r\n# 69 M 1085 1629 x1\r\n# Aldomet, Inderal\r\n'

{'Server': 'Apache/2.2.22 (Debian) mod_auth_tkt/2.1.0 mod_auth_pgsql/2.0.3 mod_ssl/2.2.22 OpenSSL/1.0.1t', 'Keep-Alive': 'timeout=5, max=100', 'Last-Modified': 'Thu, 30 Jul 1992 05:36:18 GMT', 'ETag': '"2b58628b-8f-287fdea221880"', 'Connection': 'Keep-Alive', 'Date': 'Fri, 17 Mar 2017 19:44:43 GMT', 'Accept-Ranges': 'bytes', 'Content-Length': '143'}

bytes

In [15]:
a = r.content.decode("ascii") 
a

'100 2 360 650000\r\n100.dat 212 200 11 1024 995 -22131 0 MLII\r\n100.dat 212 200 11 1024 1011 20052 0 V5\r\n# 69 M 1085 1629 x1\r\n# Aldomet, Inderal\r\n'

In [21]:
import os
import requests

def streamheader(recordname, pbdir):

    # Full url of header location
    url = 'http://physionet.org/physiobank/database/'+os.path.join(pbdir, recordname+'.hea')
    r = requests.get(url)
    
    # Raise HTTPError if invalid url
    r.raise_for_status()
        
    # Get each line as a string
    filelines = r.content.decode('ascii').splitlines()
    
    # Separate content into header and comment lines
    headerlines = []
    commentlines = []
    
    for line in filelines:
        line = line.strip()
        # Comment line
        if line.startswith('#'):  
            commentlines.append(line)
        # Non-empty non-comment line = header line.
        elif line:  
            # Look for a comment in the line
            ci = line.find('#')
            if ci > 0:
                headerlines.append(line[:ci])
                # comment on same line as header line
                commentlines.append(line[ci:])
            else:
                headerlines.append(line)
    
    return (headerlines, commentlines) 

#streamheader('3045253', 'mimic2wdb/30/3045253')
a, b = streamheader('100', 'mitdb')

print(a)

print(b)


['100 2 360 650000', '100.dat 212 200 11 1024 995 -22131 0 MLII', '100.dat 212 200 11 1024 1011 20052 0 V5']
['# 69 M 1085 1629 x1', '# Aldomet, Inderal']


In [23]:
request

<Response [404]>

In [28]:
requests.HTTPError

requests.exceptions.HTTPError

In [93]:
r = streamdat('100.dat', 'mitdb', '212', 9, 0)

print(r.status_code)

# Header the server sent
print(r.headers)

# Header we sent
print(r.request.headers)

print(len(r.content))

http://physionet.org/physiobank/database/mitdb/100.dat
{'Accept-Encoding': '*/*', 'Range': 'bytes=0-9'}
206
{'Server': 'Apache/2.2.22 (Debian) mod_auth_tkt/2.1.0 mod_auth_pgsql/2.0.3 mod_ssl/2.2.22 OpenSSL/1.0.1t', 'Date': 'Mon, 20 Mar 2017 20:58:50 GMT', 'Last-Modified': 'Thu, 30 Jul 1992 05:36:18 GMT', 'Content-Range': 'bytes 0-9/1950000', 'Accept-Ranges': 'bytes', 'Vary': 'Accept-Encoding', 'Keep-Alive': 'timeout=5, max=100', 'Content-Type': 'application/octet-stream', 'Connection': 'Keep-Alive', 'Content-Length': '10', 'ETag': '"2b58628a-1dc130-287fdea221880"'}
{'Accept': '*/*', 'User-Agent': 'python-requests/2.12.4', 'Connection': 'keep-alive', 'Range': 'bytes=0-9', 'Accept-Encoding': '*/*'}
10


In [1]:
import wfdb